In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('../input/automatic-classification-of-fetal-ultrasound-brain/data/data/Patient00188_Plane3_2_of_3.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
path_folder = "../input/automatic-classification-of-fetal-ultrasound-brain"
picture_size = 64
batch_size = 16

In [ ]:
print(img.shape)

In [ ]:
train = pd.read_csv("../input/automatic-classification-of-fetal-ultrasound-brain/train.csv")
test = pd.read_csv("../input/automatic-classification-of-fetal-ultrasound-brain/test.csv")

In [ ]:
train["image_name"] = train["image_name"].apply(lambda x : x + '.png')

In [ ]:
test["image_name"] = test["image_name"].apply(lambda x : x + '.png')

In [ ]:
train["class"].value_counts()

In [ ]:
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator 

datagen_train = ImageDataGenerator()
datagen_test = ImageDataGenerator()

train_set = datagen_train.flow_from_dataframe(directory = path_folder+"/data/data",
                                              dataframe = train,
                                              x_col = 'image_name',
                                              y_col='class',
                                              target_size=(picture_size,picture_size),
                                              batch_size = batch_size,
                                              class_mode='categorical',
                                              color_mode='grayscale',
                                              shuffle=True)

test_set = datagen_test.flow_from_dataframe(directory = path_folder+"/data/data",
                                              dataframe = test,
                                              x_col = 'image_name',
                                              target_size=(picture_size,picture_size),
                                              batch_size = batch_size,
                                              class_mode=None,
                                              color_mode='grayscale',)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Dropout, MaxPooling2D, BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
nbr_of_classes = 4
model = Sequential()

#first cnn layer
model.add(Conv2D(64,(3,3), padding='same',input_shape=(64,64,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

#2nd cnn layer
model.add(Conv2D(128,(3,3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

#3rd cnn layer 
model.add(Conv2D(512,(3,3),padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

#4th cnn layer
model.add(Conv2D(512,(3,3),padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))

#flatten layer
model.add(Flatten()) # 2d to 1d

#Fully connected 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


#output layer
model.add(Dense(nbr_of_classes, activation="sigmoid"))

model.compile(optimizer=Adam(learning_rate=0.01),metrics=["accuracy"],loss='categorical_crossentropy')
model.summary()

In [ ]:
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("Model.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001) 

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

epochs = 4

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
history = model.fit_generator(generator=train_set,
                                steps_per_epoch=train_set.n//train_set.batch_size,
                                epochs=epochs,
                                callbacks=callbacks_list
                                ) 
model.save("Modelsign6__.h5")

In [ ]:
r = model.predict(test_set)

In [ ]:
t=[]
for i in range(len(r)):
    t.append(r[i].argmax())

In [ ]:
train_set.class_indices

In [ ]:
for i in range(len(t)):
    if t[i] == 0:
        t[i] = 'Other'
    elif t[i] == 1:
        t[i] = 'Trans-cerebellum'
    elif t[i] == 2:
        t[i] = 'Trans-thalamic'
    else:
        t[i] = 'Trans-ventricular'

In [ ]:
SubmissionFile = pd.read_csv("../input/automatic-classification-of-fetal-ultrasound-brain/sample_submission.csv")

In [ ]:
SubmissionFile 

In [ ]:
SubmissionFile["class"] = t

In [ ]:
SubmissionFile.to_csv('submission.csv', index=False)